# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

# Challenge 1 - Accessing our database
Create a connection to access the sakila database with the ip, user and password provided in class. Take a look at the data. Do some joins in order to have more elaborated tables in a dataframe (you can try first the query in the DBMS and then use it in your notebook).


In [1]:
# your code here

#Let's import all neded modules
from sqlalchemy import create_engine
import pandas as pd

In [1]:
# connection string
sqlitedb_rel_path = '../data/juliarochflores.db'

conn_str = f'sqlite:///{sqlitedb_rel_path}'

engine = create_engine(conn_str)

In [3]:
# list all tables in database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)

,name
0,rank_info
1,personal_info
2,business_info


In [4]:
# query of the first
df_sqlite = pd.read_sql_query("select * from rank_info",engine)

df_sqlite.head()

,id,name,position,Unnamed: 0
0,6367,ananT ASAVABHOKHIN & family,NaN,0
1,7613,arash FERDOWSI,NaN,1
2,6139,banthoon LAMSAM & FAMILY,NaN,2
3,5671,bill ADDERLEY & family,NaN,3
4,2117,boris zINGArevich,NaN,4


# Challenge 2 - Working with JSON files

Import the pandas library.

In [1]:
# your code here

In the next cell, load the data in `nasa.json` in the `data` folder and load it into a pandas dataframe. Name the dataframe `nasa`.

In [3]:
# your code here

Now that we have loaded the data, let's examine it using the `head()` function.

In [4]:
# your code here

#### The `value_counts()` function is commonly used in pandas to find the frequency of every value in a column.

In the cell below, use the `value_counts()` function to determine the frequency of all types of asteroid landings by applying the function to the `fall` column.

In [5]:
# your code here

Finally, let's save the dataframe as a json file again. Save the dataframe using the `orient=records` argument and name the file `nasa-output.json`. Remember to save the file inside the `data` folder.

In [6]:
# your code here

# Challenge 2 - Working with CSV and Other Separated Files

CSV files are more commonly used as dataframes. In the cell below, load the file from the URL provided using the `read_csv()` function in pandas. Starting version 0.19 of pandas, you can load a CSV file into a dataframe directly from a URL without having to load the file first and then transform it. The dataset we will be using contains information about NASA shuttles.

In the cell below, we define the column names and the URL of the data. Following this cell, read the tst file to a variable called `shuttle`. Since the file does not contain the column names, you must add them yourself using the column names declared in `cols` using the `names` argument. Additionally, a tst file is space separated, make sure you pass ` sep=' '` to the function.

In [1]:
cols = ['time', 'rad_flow', 'fpv_close', 'fpv_open', 'high', 'bypass', 'bpv_close', 'bpv_open', 'class']
tst_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/shuttle/shuttle.tst'

In [8]:
# your code here

Let's verify that this worked by looking at the `head()` function.

In [9]:
# your code here

To make life easier for us, let's turn this dataframe into a comma separated file by saving it using the `to_csv()` function. Save `shuttle` into the file `shuttle.csv` and ensure the file is comma separated, that we are not saving the index column and that the file is saved inside the `data` folder.

In [10]:
# your code here

# Challenge 3 - Working with Excel Files

We can also use pandas to convert excel spreadsheets to dataframes. Let's use the `read_excel()` function. In this case, `astronauts.xls` is in the `data` folder. Read this file into a variable called `astronaut`. 

Note: Make sure to install the `xlrd` library if it is not yet installed.

In [11]:
# your code here

Use the `head()` function to inspect the dataframe.

In [12]:
# your code here

Use the `value_counts()` function to find the most popular undergraduate major among all astronauts.

In [13]:
# your code here

Due to all the commas present in the cells of this file, let's save it as a tab separated csv file. In the cell below, save `astronaut` as a **tab separated file** using the `to_csv` function. Call the file `astronaut.csv`. Remember to remove the index column and save the file in the `data` folder.

In [14]:
# your code here

# Bonus Challenge - Fertility Dataset

Visit the following [URL](https://archive.ics.uci.edu/ml/datasets/Fertility) and retrieve the dataset as well as the column headers. Determine the correct separator and read the file into a variable called `fertility`. Examine the dataframe using the `head()` function. 

Look in Google for a way to retrieve this data!

In [15]:
# your code here